# Analysis of hazardous asteroids' orbital parameters

Using data on already discovered asteroid's we'll try to analyse orbital parameters of hazardous asteroids and  find out orbits or familes of orbits with high probability of yet undiscovered potentially hazardous asteroids existance.

In [8]:
import numpy as np
import pickle, sys, os, time
import pandas as pd
import read_database as rdb


**Define constants:**

In [18]:
G = 6.67384e-11
M = 1.989e30
AU = 149597870700

### JPL ASTEROID DATABASE

**Load asteroid database:**

In [20]:
database_path = "./asteroid_data/latest_fulldb.csv"

if not os.path.exists(database_path):
    database_url = "http://www.ianww.com/latest_fulldb.csv"
    retrieve_database(database_url, database_path, verbose=True)

database = pd.read_csv(database_path, sep=',', usecols=[
                       'a', 'e', 'i', 'w', 'om', 'q', 'H', 'neo',
                       'pha', 'moid', 'per', 'n', 'ma', 'epoch'], 
                       low_memory=False)

db_head = database[:5]
print "db_head:\n", db_head

db_head:
  neo pha     H      epoch         e         a         q          i  \
0   N   N  3.34  2457200.5  0.075783  2.767972  2.558208  10.592305   
1   N   N  4.13  2457200.5  0.231158  2.772004  2.131234  34.840258   
2   N   N  5.33  2457200.5  0.255830  2.671073  1.987732  12.986619   
3   N   N  3.20  2457200.5  0.088834  2.361913  2.152095   7.140096   
4   N   N  6.85  2457200.5  0.191188  2.573480  2.081462   5.368549   

           om           w          ma         n          per     moid  
0   80.327198   72.654146  138.662175  0.214023  1682.058334  1.59427  
1  173.092151  309.965564  120.919026  0.213557  1685.734286  1.23108  
2  169.860102  248.399241   78.215743  0.225775  1594.509751  1.03987  
3  103.849635  151.188504   75.172676  0.271524  1325.849757  1.13975  
4  141.595776  358.969279  307.898284  0.238739  1507.924540  1.09509  


**Extract NEOs:**

In [6]:
neo, num_neo = rdb.get_neo(database)

**Recalculate MOID using orbital parameters:**

In [21]:
#rdb.calc_moid(neo, verbose=True)

**Split NEOs by groups:**

In [11]:
apollos, num_apollos = rdb.get_apollos(neo)
atens, num_atens = rdb.get_atens(neo)
amors, num_amors = rdb.get_amors(neo)

# print "apollos:", num_apollos, "neo:", num_neo, "ratio:", num_apollos/float(num_neo)
# print "atens:", num_atens, "neo:", num_neo, "ratio:", num_atens/float(num_neo)
# print "amors:", num_amors, "neo:", num_neo, "ratio:", num_amors/float(num_neo)

**Cut off apollo asteroids with extreme orbital parameters:**

In [12]:
apollos, num_apollos = rdb.cutoff_outcasts(apollos)

## GENERATE RANDOM ASTEROIDS